In [1]:
%load_ext autoreload
%autoreload
from factory import *
import torch
import os 
import sys
from catalyst.dl.callbacks import CriterionCallback, EarlyStoppingCallback, OptimizerCallback, CriterionAggregatorCallback, F1ScoreCallback, AUCCallback
from catalyst.dl.runner import SupervisedRunner
from pytorch_toolbelt import losses as L
from pytorch_toolbelt.inference import tta
import collections
from pytorch_toolbelt.utils.catalyst import * 
from metrics import *
import matplotlib.pyplot as plt
from viz_utils import *
from tqdm import tqdm
import cv2
import segmentation_models_pytorch as smp
from catalyst.contrib.optimizers import RAdam, Lookahead
%matplotlib inline

pyarrow not available, switching to pickle. To install pyarrow, run `pip install pyarrow`.
lz4 not available, disabling compression. To install lz4, run `pip install lz4`.


In [2]:
#encoder_names = ['efficientnet-b5',
#            'efficientnet-b7',
#            'se_resnext50_32x4d',
#            'se_resnext101_32x4d']
#experiment_names = [
#            'efficientnet-b5_simple_posttrain',
#            'efficientnet-b7_simple_posttrain',
#            'se_resnext50_32x4d_simple_posttrain',
#            'se_resnext101_32x4d_simple_posttrain']
#log_dirs = [
#            'logs/efficientnet-b5_simple_posttrain',
#            'logs/efficientnet-b7_simple_posttrain',
#            'logs/se_resnext50_32x4d_simple_posttrain',
#            'logs/se_resnext101_32x4d_simple_posttrain']
#output_channels = [4,4,4,4]
experiment_names = [
    'se_resnext50_32x4d_simple',
    'se_resnext101_32x4d_simple',
    'efficientnet-b5_simple',
    'resnext50_32x4d_with_mask_and_boundaries', 
    'resnext101_32x8d_with_mask_and_boundaries', 
    'se_resnet50_simple',
    'resnext50_32x4d_simple']
log_dirs = [
    'logs/se_resnext50_32x4d_simple',
    'logs/se_resnext101_32x4d_simple',
    'logs/efficientnet-b5_simple',
    'logs/resnext50_32x4d_with_mask_and_boundaries', 
    'logs/resnext101_32x8d_with_mask_and_boundaries', 
    'logs/se_resnet50_simple',
    'logs/resnext50_32x4d_simple']
encoder_names = [
    'se_resnext50_32x4d',
    'se_resnext101_32x4d',
    'efficientnet-b5',
    'resnext50_32x4d', 
    'resnext101_32x8d', 
    'se_resnet50', 
    'resnext50_32x4d']
output_channels = [4,4,4,6,6,4,4]
train_df_path = 'data/train.csv'
data_folder = "data/train_images/"
test_data_folder = "data/test_images/"
val_output_folder = "data/validation_predictions"
sample_submission_path = 'data/sample_submission.csv'

In [3]:
class TestDataset(Dataset):
    '''Dataset for test prediction'''
    def __init__(self, root, fnames, transforms):
        self.root = root
        self.fnames = fnames.tolist()
        self.num_samples = len(self.fnames)
        self.transforms = transforms

    def __getitem__(self, idx):
        fname = self.fnames[idx]
        path = os.path.join(self.root, fname)
        image = cv2.imread(path)
        image = self.transforms(image=image)["image"]
        image = torch.from_numpy(image.transpose((2, 0, 1))).float()
        return {'features':image, 'image_name':fname}

    def __len__(self):
        return self.num_samples

In [5]:
thr = [0.55, 0.5, 0.55, 0.40]
min_size = [650, 500, 1200, 2100]

In [6]:
sample_submission = pd.read_csv(sample_submission_path)
test_images = np.unique([x.split('_')[0] for x in sample_submission.ImageId_ClassId.values])
test_loader = DataLoader(
    TestDataset(test_data_folder, test_images, validation_augmentations()),
    batch_size=8,
    num_workers=12,
    shuffle=False,   
    )

In [7]:
predictions_test = []
for idx, exp_name in enumerate(experiment_names):
    print('Infering test using {}'.format(exp_name))
    runner = SupervisedRunner(input_key='features')
    model = smp.Unet(classes=output_channels[idx], 
                    encoder_name=encoder_names[idx],
                    encoder_weights=None)
    predictions_test.append(runner.predict_loader(
            model=model,
            loader=test_loader,
            resume=f"{log_dirs[idx]}/checkpoints/best.pth",
            verbose=True,
        ))

Infering test using se_resnext50_32x4d_simple
=> loading checkpoint logs/se_resnext50_32x4d_simple/checkpoints/best.pth
loaded checkpoint logs/se_resnext50_32x4d_simple/checkpoints/best.pth (epoch 78)
0/1 * Epoch (infer): 100% 226/226 [01:30<00:00,  2.51it/s, _timers/_fps=340.557]
Top best models:

Infering test using se_resnext101_32x4d_simple
=> loading checkpoint logs/se_resnext101_32x4d_simple/checkpoints/best.pth
loaded checkpoint logs/se_resnext101_32x4d_simple/checkpoints/best.pth (epoch 47)
0/1 * Epoch (infer): 100% 226/226 [01:57<00:00,  1.92it/s, _timers/_fps=223.877]
Top best models:

Infering test using efficientnet-b5_simple
=> loading checkpoint logs/efficientnet-b5_simple/checkpoints/best.pth
loaded checkpoint logs/efficientnet-b5_simple/checkpoints/best.pth (epoch 39)
0/1 * Epoch (infer): 100% 226/226 [02:11<00:00,  1.72it/s, _timers/_fps=288.479]
Top best models:

Infering test using resnext50_32x4d_with_mask_and_boundaries
=> loading checkpoint logs/resnext50_32x4d_wi

In [8]:
images_id = []
masks_rle = []
for idx in tqdm(range(len(test_images))):
    image_pred = np.stack([nn.Sigmoid()(torch.from_numpy(data_[idx][:4,...])).numpy() for data_ in predictions_test]).mean(axis=0)
    for defect_type in range(4):
        image_pred_bin = image_pred[defect_type,...]>thr[defect_type]
        if image_pred_bin.sum() < min_area[defect_type]:
            masks_rle.append('')
        else:
            masks_rle.append(mask2rle(image_pred_bin))
        images_id.append(test_images[idx]+'_{}'.format(defect_type+1))     

100%|██████████| 1801/1801 [00:48<00:00, 36.86it/s]


In [9]:
submission = pd.DataFrame({'ImageId_ClassId':images_id,'EncodedPixels':masks_rle})
classifications = pd.read_csv('classificaion_test/effnetb3_3folds.csv')
classifications.columns.values[1] = 'has_mask'
submission = submission.merge(classifications)
submission.loc[submission.has_mask==0, 'EncodedPixels'] = ''
submission.to_csv('./submission_finalBlendV3_simplePosprocess.tsv')

Now a different inference

In [ ]:
def postprocess(probability, cls):
    min_sizes = [600, 600, 1000, 2000]
    min_size = min_sizes[cls-1]
    thresholds = [0.5, 0.5, 0.5, 0.5]
    threshold = thresholds[cls-1]
    mask = cv2.threshold(probability, threshold, 1, cv2.THRESH_BINARY)[1]
    num_component, component = cv2.connectedComponents(mask.astype(np.uint8))
    predictions = np.zeros((256, 1600), np.float32)
    num = 0
    for c in range(1, num_component):
        p = (component == c)
        if p.sum() > min_size:
            predictions[p] = 1
            num += 1
    return predictions, num

In [ ]:
def load_model(idx, classes=4):
    model = smp.Unet(encoder_names[idx], 
                     encoder_weights=None, 
                     classes=output_channels[idx], 
                     activation="sigmoid")
    model.load_state_dict(torch.load(os.path.join(log_dirs[idx],'checkpoints/best.pth'))['model_state_dict'])
    model.cuda()
    model.eval()
    return model
models = [load_model(idx) for idx in range(len(encoder_names))]

In [ ]:
predictions = []
for i, batch in enumerate(tqdm(test_loader)):
    fnames = batch['image_name']
    images = batch['features']
    res = []
    with torch.no_grad():
        for model in models:
            batch_preds = torch.sigmoid(model(images.cuda()))
            res.append(batch_preds[:,:4,...])
    res = torch.stack(res)
    batch_preds = torch.mean(res, dim=0)
    batch_preds = batch_preds.detach().cpu().numpy()

    for fname, preds in zip(fnames, batch_preds):
        #fname_defects = defect_dict[fname]
        #no_defect_types = [i for i in [1, 2, 3, 4] if i not in fname_defects]
        #no_defect_types = [i for i in [1, 2, 3, 4]]
        #for defect_type in no_defect_types:
        #    preds[defect_type - 1] = np.zeros(shape=(256, 1600))

        for cls, pred in enumerate(preds):
            pred, num = postprocess(pred, cls + 1)
            rle = mask2rle(pred)
            name = fname + f"_{cls + 1}"
            predictions.append([name, rle])

In [ ]:
submission = pd.DataFrame({'ImageId_ClassId':[x[0] for x in predictions],'EncodedPixels':[x[1] for x in predictions]})
classifications = pd.read_csv('classificaion_test/effnetb3_3folds.csv')
classifications.columns.values[1] = 'has_mask'
submission = submission.merge(classifications)
submission.loc[submission.has_mask==0, 'EncodedPixels'] = ''
submission.to_csv('./submission_finalBlendV2_newPosprocess.tsv')

In [ ]:
.shape

In [ ]:
batch_preds.sum()